<a href="https://colab.research.google.com/github/rachit2005/Large-Language-Model/blob/main/llm_tokenizer_%3E_tiktokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# using unicode byte encoding in utf-8

list("hello world".encode("utf-8")) # --> these are byte stream

[104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100]

# Encoder (Vocabulary Construction):
1. Initialization:
The encoder begins by initializing the vocabulary with individual characters (or bytes) of the text.
2. Iterative Merging:
The encoder then iteratively merges the most frequent pairs of characters (or subwords) in the vocabulary.
3. Vocabulary Expansion:
This merging process expands the vocabulary with new subword units until a desired vocabulary size is reached.
4. Vocabulary Storage:
The final vocabulary of subwords, along with their corresponding merge rules, is stored for later use in the decoder.  

In [2]:
# byte pair encoding --> https://en.wikipedia.org/wiki/Byte-pair_encoding


# 1. Initialization

text = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum."
tokens = text.encode("utf-8")

tokens = list(map(int , tokens))

print(tokens)

print(len(text))
print(len(tokens))


[76, 111, 114, 101, 109, 32, 73, 112, 115, 117, 109, 32, 105, 115, 32, 115, 105, 109, 112, 108, 121, 32, 100, 117, 109, 109, 121, 32, 116, 101, 120, 116, 32, 111, 102, 32, 116, 104, 101, 32, 112, 114, 105, 110, 116, 105, 110, 103, 32, 97, 110, 100, 32, 116, 121, 112, 101, 115, 101, 116, 116, 105, 110, 103, 32, 105, 110, 100, 117, 115, 116, 114, 121, 46, 32, 76, 111, 114, 101, 109, 32, 73, 112, 115, 117, 109, 32, 104, 97, 115, 32, 98, 101, 101, 110, 32, 116, 104, 101, 32, 105, 110, 100, 117, 115, 116, 114, 121, 39, 115, 32, 115, 116, 97, 110, 100, 97, 114, 100, 32, 100, 117, 109, 109, 121, 32, 116, 101, 120, 116, 32, 101, 118, 101, 114, 32, 115, 105, 110, 99, 101, 32, 116, 104, 101, 32, 49, 53, 48, 48, 115, 44, 32, 119, 104, 101, 110, 32, 97, 110, 32, 117, 110, 107, 110, 111, 119, 110, 32, 112, 114, 105, 110, 116, 101, 114, 32, 116, 111, 111, 107, 32, 97, 32, 103, 97, 108, 108, 101, 121, 32, 111, 102, 32, 116, 121, 112, 101, 32, 97, 110, 100, 32, 115, 99, 114, 97, 109, 98, 108, 101, 100

In [3]:
# finding the pair of bytes that occur most frequently

def get_pair_freq(tokens):
  pair_freq = {}
  for pair in zip(tokens, tokens[1:]):
    pair_freq[pair] = pair_freq.get(pair , 0) + 1
  return pair_freq

stats = get_pair_freq(tokens)
print(sorted(((v,k) for k,v in stats.items()) , reverse=True))
top_pair = max(stats , key = stats.get)
print(top_pair)

[(17, (105, 110)), (15, (101, 32)), (14, (32, 116)), (9, (115, 32)), (9, (114, 101)), (8, (116, 104)), (8, (116, 32)), (8, (110, 103)), (8, (104, 101)), (8, (32, 115)), (7, (121, 32)), (7, (110, 116)), (7, (109, 32)), (7, (100, 32)), (7, (32, 105)), (7, (32, 97)), (6, (117, 109)), (6, (115, 101)), (6, (110, 100)), (6, (110, 32)), (6, (103, 32)), (6, (101, 115)), (6, (101, 110)), (6, (97, 115)), (6, (97, 110)), (6, (32, 73)), (5, (115, 117)), (5, (112, 101)), (5, (111, 114)), (5, (111, 102)), (5, (108, 101)), (5, (101, 116)), (5, (101, 109)), (5, (100, 117)), (5, (32, 112)), (5, (32, 111)), (4, (121, 112)), (4, (118, 101)), (4, (116, 121)), (4, (116, 114)), (4, (116, 111)), (4, (116, 105)), (4, (114, 105)), (4, (112, 115)), (4, (111, 110)), (4, (108, 121)), (4, (102, 32)), (4, (101, 114)), (4, (101, 100)), (4, (76, 111)), (4, (73, 112)), (4, (44, 32)), (4, (32, 119)), (4, (32, 76)), (3, (117, 115)), (3, (116, 101)), (3, (115, 116)), (3, (115, 105)), (3, (115, 44)), (3, (114, 32)), (3, (

In [4]:
# iterative merging --> merging the top_pair

def merge(ids, pair ,idx):
  new_ids = []
  i = 0

  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      new_ids.append(idx)
      i +=2
    else:
      new_ids.append(ids[i])
      i +=1

  return new_ids

# print(merge([5,6,6,7,9,1] , (6,7) , 99))

tokens2 = merge(tokens , top_pair , 256)
print(tokens2 , f"len : {len(tokens2)}")

[76, 111, 114, 101, 109, 32, 73, 112, 115, 117, 109, 32, 105, 115, 32, 115, 105, 109, 112, 108, 121, 32, 100, 117, 109, 109, 121, 32, 116, 101, 120, 116, 32, 111, 102, 32, 116, 104, 101, 32, 112, 114, 256, 116, 256, 103, 32, 97, 110, 100, 32, 116, 121, 112, 101, 115, 101, 116, 116, 256, 103, 32, 256, 100, 117, 115, 116, 114, 121, 46, 32, 76, 111, 114, 101, 109, 32, 73, 112, 115, 117, 109, 32, 104, 97, 115, 32, 98, 101, 101, 110, 32, 116, 104, 101, 32, 256, 100, 117, 115, 116, 114, 121, 39, 115, 32, 115, 116, 97, 110, 100, 97, 114, 100, 32, 100, 117, 109, 109, 121, 32, 116, 101, 120, 116, 32, 101, 118, 101, 114, 32, 115, 256, 99, 101, 32, 116, 104, 101, 32, 49, 53, 48, 48, 115, 44, 32, 119, 104, 101, 110, 32, 97, 110, 32, 117, 110, 107, 110, 111, 119, 110, 32, 112, 114, 256, 116, 101, 114, 32, 116, 111, 111, 107, 32, 97, 32, 103, 97, 108, 108, 101, 121, 32, 111, 102, 32, 116, 121, 112, 101, 32, 97, 110, 100, 32, 115, 99, 114, 97, 109, 98, 108, 101, 100, 32, 105, 116, 32, 116, 111, 32, 1

In [5]:
# it will iterate until the desired vocab size is reached

vocab_size = 276 # the desired vocab size
num_merges = vocab_size - 256
lds = list(tokens) # copuy and do not destory the og

merges = {} # (int , int) = int

for i in range(num_merges):
  stats = get_pair_freq(lds)
  top_pair = max(stats , key = stats.get)
  idx = 256 + i
  print(f"merging : {top_pair} into a new token {idx}")
  merges[top_pair] = idx
  lds = merge(lds , top_pair , idx)


print(lds)
print(len(lds))

print(f"vocab size = {len(set(lds))}")

merging : (105, 110) into a new token 256
merging : (101, 32) into a new token 257
merging : (32, 116) into a new token 258
merging : (115, 32) into a new token 259
merging : (114, 101) into a new token 260
merging : (109, 32) into a new token 261
merging : (116, 32) into a new token 262
merging : (256, 103) into a new token 263
merging : (104, 257) into a new token 264
merging : (263, 32) into a new token 265
merging : (97, 110) into a new token 266
merging : (101, 115) into a new token 267
merging : (101, 110) into a new token 268
merging : (100, 32) into a new token 269
merging : (115, 117) into a new token 270
merging : (121, 32) into a new token 271
merging : (100, 117) into a new token 272
merging : (111, 102) into a new token 273
merging : (258, 264) into a new token 274
merging : (108, 101) into a new token 275
[76, 111, 260, 261, 73, 112, 270, 261, 105, 259, 115, 105, 109, 112, 108, 271, 272, 109, 109, 121, 258, 101, 120, 262, 273, 274, 112, 114, 256, 116, 265, 266, 100, 258, 

In [6]:
print(f"token list at start = {len(tokens)}")
print(f" final len = {len(lds)}")
print(f"compression ratio : {len(tokens)/len(lds)}X")

token list at start = 574
 final len = 426
compression ratio : 1.3474178403755868X


In [7]:
def encode(text:str):
  tokens = list(text.encode("utf-8"))
  while True:
    stats = get_pair_freq(tokens)
    if not stats:
      break
    pair = min(stats , key = lambda p: merges.get(p , float("inf")))
    if pair not in merges:
      break
    idx = merges[pair]
    tokens = merge(tokens , pair , idx)

  return tokens

print(encode("heloo world"))

[104, 101, 108, 111, 111, 32, 119, 111, 114, 108, 100]


# Decoder (Tokenization):
1. Lookup Table:
The decoder uses the vocabulary created by the encoder as a lookup table.
2. Subword Decomposition:
When decoding a new piece of text, the decoder searches the vocabulary for the longest subword that matches the current prefix of the input text.
3. Tokenization:
The decoder then replaces the matched subword with the corresponding token from the vocabulary.
4. Iteration:
This process is repeated until the entire input text has been broken down into a sequence of tokens.

In [8]:
# merges --> lookup table for decoding the subwords into the real worlds

vocab = {idx : bytes([idx]) for idx in range(256)}
for (p0 , p1) , idx in sorted(merges.items() , key= lambda x : x[1]):
  vocab[idx] = vocab[p0] + vocab[p1]

# print(list(vocab))

def decode(lds):
  "given lds (list of integers) --> python string"
  tokens = b"".join(vocab[idx] for idx in lds)
  text = tokens.decode("utf-8" , errors="replace")
  return text

text2 = decode(lds)
print(text2 == text)

print(decode([256]))
print(decode(encode("hello world")))

True
in
hello world


full working mode and summary --> remember the code is beutify by the chatgpt

In [9]:
# ---------------------------------------
# UTILITY: Find the most frequent adjacent byte pairs
# ---------------------------------------
def get_pair_freq(tokens):
  pair_freq = {}
  for pair in zip(tokens, tokens[1:]):
    pair_freq[pair] = pair_freq.get(pair , 0) + 1
  return pair_freq


# ---------------------------------------
# UTILITY: Merge the most frequent pair into a new token ID
# ---------------------------------------
def merge(ids, pair, idx):
  new_ids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      new_ids.append(idx)  # merge the pair into a new token
      i += 2
    else:
      new_ids.append(ids[i])
      i += 1
  return new_ids


# =====================================================
# ⬇️ TRAINING PHASE: Learn merges from initial text ⬇️
# =====================================================

vocab_size = 276                # desired vocabulary size
num_merges = vocab_size - 256  # how many merges we will do
lds = list(tokens)             # input byte token list (training data)

merges = {}  # dictionary: (pair of ints) -> new token index

# 🔁 Iteratively perform merges to build BPE vocabulary
for i in range(num_merges):
  stats = get_pair_freq(lds)
  if not stats:
    break
  top_pair = max(stats, key=stats.get)
  idx = 256 + i  # new token index starts at 256
  print(f"merging : {top_pair} into a new token {idx}")
  merges[top_pair] = idx
  lds = merge(lds, top_pair, idx)  # apply the merge

# ==========================================================
# ⬇️ VOCABULARY CONSTRUCTION: Map token IDs to byte strings ⬇️
# ==========================================================

# base vocab: 0–255 → single byte values
vocab = {idx: bytes([idx]) for idx in range(256)}

# apply merges in order to build up the new tokens
for (p0, p1), idx in sorted(merges.items(), key=lambda x: x[1]):
  vocab[idx] = vocab[p0] + vocab[p1]

# =====================================================
# ⬇️ ENCODING FUNCTION: Text → List of merged token IDs ⬇️
# =====================================================
def encode(text: str):
  tokens = list(text.encode("utf-8"))  # convert to byte list
  while True:
    stats = get_pair_freq(tokens)
    if not stats:
      break
    # pick the most prioritary pair (lowest merge index)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens


# ====================================================
# ⬇️ DECODING FUNCTION: List of token IDs → String ⬇️
# ====================================================
def decode(lds):
  tokens = b"".join(vocab[idx] for idx in lds)
  text = tokens.decode("utf-8", errors="replace")
  return text

# ================================
# ⬇️ Example Usage / Testing ⬇️
# ================================
print(encode("heloo world"))   # encoded token ID list
print(decode([104, 101, 108, 111, 111, 32, 119, 111, 114, 108, 100]))  # decoded from bytes
print(decode(encode("hello world")))  # full round-trip test


merging : (105, 110) into a new token 256
merging : (101, 32) into a new token 257
merging : (32, 116) into a new token 258
merging : (115, 32) into a new token 259
merging : (114, 101) into a new token 260
merging : (109, 32) into a new token 261
merging : (116, 32) into a new token 262
merging : (256, 103) into a new token 263
merging : (104, 257) into a new token 264
merging : (263, 32) into a new token 265
merging : (97, 110) into a new token 266
merging : (101, 115) into a new token 267
merging : (101, 110) into a new token 268
merging : (100, 32) into a new token 269
merging : (115, 117) into a new token 270
merging : (121, 32) into a new token 271
merging : (100, 117) into a new token 272
merging : (111, 102) into a new token 273
merging : (258, 264) into a new token 274
merging : (108, 101) into a new token 275
[104, 101, 108, 111, 111, 32, 119, 111, 114, 108, 100]
heloo world
hello world


# Forced splits using regex pattern

In [10]:
import regex as re

gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""" , flags=re.IGNORECASE)

print(re.findall(gpt2pat , "Hello World HOW'S are YOU 2342            "))

['Hello', ' World', ' HOW', "'S", ' are', ' YOU', ' 2342', '            ']


In [11]:
eg = """for i in range(1, 6):
  for j in range(i):
    print("*", end="")
  print()"""

print(re.findall(gpt2pat , eg))

['for', ' i', ' in', ' range', '(', '1', ',', ' 6', '):', '\n ', ' for', ' j', ' in', ' range', '(', 'i', '):', '\n   ', ' print', '("*",', ' end', '="")', '\n ', ' print', '()']


# Building the GPT-4 Tokenizer

https://github.com/karpathy/minbpe/blob/master/exercise.md

In [12]:
!wget "https://raw.githubusercontent.com/karpathy/minbpe/refs/heads/master/tests/taylorswift.txt"

--2025-06-12 14:35:38--  https://raw.githubusercontent.com/karpathy/minbpe/refs/heads/master/tests/taylorswift.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 185768 (181K) [text/plain]
Saving to: ‘taylorswift.txt’

taylorswift.txt     100%[===================>] 181.41K  --.-KB/s    in 0.02s   

2025-06-12 14:35:38 (8.05 MB/s) - ‘taylorswift.txt’ saved [185768/185768]



In [13]:
# step 1

class BasicTokenizer():
  def __init__(self  , special_tokens:list = None) -> None:
    self.vocab_size = 0
    self.verbose = False
    self.special_tokens = special_tokens or []
    self.merges = {} # --> {(int , int) : int}
    self.vocab = {} # --> {token_str : token_id}
    self.vocab_inv = {} # --> {token_id : token_str}
    # GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
    self.patterns = re.compile(r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+""" , flags=re.IGNORECASE)

  def get_pair_freq(self, tokens):
    pairs = {}
    for pair in zip(tokens , tokens[1:]):
      pairs[pair] = pairs.get(pair , 0) + 1

    return pairs

  def merge(self , ids, pair, idx):
    new_ids = []
    i = 0
    while i < len(ids):
      if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
        new_ids.append(idx)  # merge the pair into a new token
        i += 2
      else:
        new_ids.append(ids[i])
        i += 1
    return new_ids

  def train(self , text:str , vocab_size:int, verbose:bool = False):
    self.vocab_size = vocab_size
    self.verbose = verbose

    words = re.findall(self.patterns , text)
    tokens = [c for word in words for c in list(word) + ["<W>"]]
    tokens = list("".join(tokens).encode("utf-8"))

    next_idx = 256

    self.vocab = {idx : bytes([idx]) for idx in range(next_idx)}

    tokens_copy = list(tokens)
    # finding the most frequent pairs
    num_merges = vocab_size - 256
    for i in range(num_merges):
      stats = self.get_pair_freq(tokens)
      if not stats:
        break
      max_freq_pair = max(stats , key=stats.get)
      next_idx +=  1
      if self.verbose:
        print(f"merging : {max_freq_pair} --> {next_idx}")

      self.merges[max_freq_pair] = next_idx
      self.vocab[next_idx] = self.vocab[max_freq_pair[0]] + self.vocab[max_freq_pair[1]]
      tokens = self.merge(tokens , max_freq_pair , next_idx)

    self.vocab_inv = {idx : self.vocab[idx] for idx in self.vocab}

  def encode(self , text:str):
      words = re.findall(self.patterns , text)
      tokens = [c for word in words for c in list(word)]
      tokens = list("".join(tokens).encode("utf-8"))
      while True:
        stats = self.get_pair_freq(tokens)
        if not stats:
          break
        # pick the most prioritary pair (lowest merge index)
        pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
        if pair not in self.merges:
          break
        idx = self.merges[pair]
        tokens = self.merge(tokens, pair, idx)
      return tokens

  def decode(self,lds):
      tokens = b"".join(self.vocab[idx] for idx in lds)
      text = tokens.decode("utf-8", errors="replace")
      return text


In [14]:
tokenizer = BasicTokenizer()
with open("taylorswift.txt" , "r") as f:
  text = f.read()

# tokenizer.train("""Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.""" , 50257)
tokenizer.train(text , 50257)

tokens = tokenizer.encode("hello world!!!? (안녕하세요!) lol123 😉")
print(tokens)

decoded_text = tokenizer.decode(tokens)
print(decoded_text)

[104, 665, 111, 32, 4144, 2801, 102, 32, 2626, 341, 116, 32, 523, 32, 609, 1897, 32, 3843, 121, 267, 32, 6503, 39, 115, 32, 3237]
hello myself Rachit and like Talyor Swift's songs


In [20]:
# match this
import tiktoken
enc = tiktoken.get_encoding("cl100k_base") # this is the GPT-4 tokenizer
ids = enc.encode("hello world!!!? (안녕하세요!) lol123 😉")
text = enc.decode(ids) # get the same text back


tokens = tokenizer.encode("hello world!!!? (안녕하세요!) lol123 😉")
# print(tokens)

decoded_text = tokenizer.decode(tokens)
print(decoded_text == text)

True
